In [1]:
import sys
import pandas as pd
import dateutil
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut

import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
tf.config.experimental_run_functions_eagerly(True)

sys.path.append('../')
from utilities.ANN_functions import root_mean_squared_error, r2_keras_loss, r2_keras
from utilities.ANN_functions import create_loso_model, create_loyo_model, create_lsyso_model

import warnings
warnings.filterwarnings('ignore')
date_parser = lambda x: dateutil.parser.parse(x, ignoretz=True)

from dask.distributed import Client
client = Client()
#client = Client(n_workers=2, threads_per_worker=1, memory_limit='1GB')
client

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:44687/status,
Dashboard: http://127.0.0.1:44687/status,Workers: 8
Total threads: 32,Total memory: 31.27 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42937,Workers: 8
Dashboard: http://127.0.0.1:44687/status,Total threads: 32
Started: Just now,Total memory: 31.27 GiB
Comm: tcp://127.0.0.1:43301,Total threads: 4
Dashboard: http://127.0.0.1:44077/status,Memory: 3.91 GiB
Nanny: tcp://127.0.0.1:33669,


In [2]:
# Load data
cs_file = '../data/SMB_input_2011_2016_ERA5.csv'

df = pd.read_csv(cs_file,
   delimiter='\t', index_col=['Date/Time'],
    parse_dates=['Date/Time'], na_values='NAN',date_parser=date_parser)

In [3]:
X = (df.drop(['Diff', 'Event'], axis=1)).to_numpy()
y = (df['Diff'].copy()).to_numpy()

In [4]:
loso = LeaveOneGroupOut()
groups = df['Event'].values

In [5]:
x, n_features = X.shape

In [6]:
path_ann = '../ANN/LOSO_ERA5/CV/'

In [7]:
def run_loso():
    # Leave-One-Stake-Out
    i = 1
    test_rmse = []
    for train_index, test_index in loso.split(X, y, groups):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model = create_loso_model(n_features, final=False)
        es = EarlyStopping(monitor='val_loss', mode='min', min_delta=0.01, patience=1000)
        mc = ModelCheckpoint(path_ann + 'stake_'+str(i)+'_model.h5', monitor='val_loss', mode='min',
                             save_best_only=True, verbose=1)

        history = model.fit(X_train, y_train, validation_data = (X_test, y_test), 
                            epochs=2000, batch_size = 11, verbose=1, callbacks=[es, mc])

        best_model = load_model(path_ann  + 'stake_'+str(i)+'_model.h5',
                                custom_objects={"r2_keras": r2_keras, 
                                                "r2_keras_loss": r2_keras_loss, 
                                                "root_mean_squared_error": root_mean_squared_error})

        score = best_model.evaluate(X_test, y_test)

        test_rmse.append(score[0])

        i= i+1
    return test_rmse

In [ ]:
x = client.submit(run_loso)
RMSE_r = pd.DataFrame(data={'RMSE': x.result()})
RMSE_r.to_csv('../out/RMSE_ANN_stakes_ERA5.csv', sep = ' ', index=False)
print(x.result())

In [3]:
df['Year'] = df.index.year

In [4]:
loyo = LeaveOneGroupOut()
groups = df['Year'].values

In [5]:
X = (df.drop(['Diff', 'Event', 'Year'], axis=1)).to_numpy()
y = (df['Diff'].copy()).to_numpy()

In [6]:
x, n_features = X.shape

In [7]:
path_ann = '../ANN/LOYO_ERA5/CV/'

In [8]:
def run_loyo():
    # Leave-One-Year-Out
    i = 1
    test_rmse = []
    for train_index, test_index in loyo.split(X, y, groups):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model = create_loyo_model(n_features)
        es = EarlyStopping(monitor='val_loss', mode='min', min_delta=0.01, patience=1000)
        mc = ModelCheckpoint(path_ann + 'year_'+str(i)+'_model.h5', monitor='val_loss', mode='min',
                             save_best_only=True, verbose=1)

        history = model.fit(X_train, y_train, validation_data = (X_test, y_test), 
                            epochs=2000, batch_size = 5, verbose=1, callbacks=[es, mc])

        best_model = load_model(path_ann  + 'year_'+str(i)+'_model.h5',
                                custom_objects={"r2_keras": r2_keras, 
                                                "r2_keras_loss": r2_keras_loss, 
                                                "root_mean_squared_error": root_mean_squared_error})

        score = best_model.evaluate(X_test, y_test)

        test_rmse.append(score[0])
        i= i+1
    return test_rmse

In [ ]:
x = client.submit(run_loyo)
RMSE_r = pd.DataFrame(data={'RMSE': x.result()})
RMSE_r.to_csv('../out/RMSE_ANN_years_ERA5.csv', sep = ' ', index=False)
print(x.result())

In [8]:
X = (df.drop(['Diff', 'Event', 'Year'], axis=1)).to_numpy()
y = (df['Diff'].copy()).to_numpy()

In [9]:
groups = df['Event'].values
years = df.drop_duplicates(subset=['Year'])['Year'].values

In [10]:
x, n_features = X.shape

In [11]:
path_ann = '../ANN/LOYSO_ERA5/CV/'

In [12]:
def run_loyso():
    # Leave-One-Year-Stake-Out
    i = 1 
    test_rmse = []
    for year, (train_index, test_index) in zip(years, loso.split(X, y, groups)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model = create_lsyso_model(n_features, final=False)
        es = EarlyStopping(monitor='val_loss', mode='min', min_delta=0.01, patience=1000)
        mc = ModelCheckpoint(path_ann + 'year_stake_'+str(i)+'_model.h5', monitor='val_loss', mode='min',
                             save_best_only=True, verbose=1)

        history = model.fit(X_train, y_train, validation_data = (X_test, y_test), 
                            epochs=2000, batch_size = 5, verbose=1, callbacks=[es, mc])

        best_model = load_model(path_ann  + 'year_stake_'+str(i)+'_model.h5',
                                custom_objects={"r2_keras": r2_keras, 
                                                "r2_keras_loss": r2_keras_loss, 
                                                "root_mean_squared_error": root_mean_squared_error})

        score = best_model.evaluate(X_test, y_test)

        test_rmse.append(score[0])
        i= i+1
    return test_rmse

In [ ]:
x = client.submit(run_loyso)
RMSE_r = pd.DataFrame(data={'RMSE': x.result()})
RMSE_r.to_csv('../out/RMSE_ANN_years_stakes_ERA5.csv', sep = ' ', index=False)
print(x.result())